In [1]:
import numpy as np
import sys
import os.path
import glob
import random
from scipy.stats import lognorm, norm
from matplotlib import pyplot as plt
import mplhep as hep 
import pyhf
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
plt.style.use('loca_v_1_5/plot_style.txt')
from loca_v_1_5.loca import *
t0=timer()
plt.style.use(hep.style.CMS)
import yaml
import time


In [2]:
def ta_vt_ATLAS_search(lhco_file, mT_min=0.0):

    process='p p --> τ v'

    search_description='''Recast of ATLAS search arXiv:2002.12223: 
    'SEARCH FOR HEAVY HIGGS BOSONS DECAYING INTO TWO TAU LEPTONS WITH
    THE ATLAS DETECTOR USING pp COLLISIONS AT SQRT(S)=13 TEV'
    Phys. Rev. Lett. 125 (2020) 051801'''

    #...Pre-cuts:
    basic_cuts = {                    
            'e'     : [15.0 , 1e+10 ,  2.47],   #[pt_min, pt_max, abs(eta_max)] 
            'mu'    : [7.0  , 1e+10 ,  2.5],
            'tau'   : [65.0 , 1e+10 ,  2.5],
            'jet'   : [20.0 , 1e+10 ,  2.5],
            'bjet'  : [20.0 , 1e+10 ,  2.5],
            'met'   : [0.0  , 1e+10 ,  1e+10],
            'photon': [0.0  , 1e+10 ,  1e+10] 
             }

    signals=[]; effs=[]; xsecs=[]

    #...Signal region defintion:
    
    Category_τhad = category(
            'nleps = 0',
            'ntaus >= 1',
            'met > 150',
            '|dphi(τ,met)| > 2.4',
            '0.7 < pT/met < 1.3',
            'mT > '+str(mT_min)
             )

    
    #...Observables:
    
    bins=[200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1750, 2000, 10000]
    MT_spectrum=spectrum(name='MT')
     
    #...Apply cuts:
    
    lhco=open_lhco(file=lhco_file, cuts=basic_cuts, lhco_header=True)

    with lhco as events:   
        for N, event in events:

            Category_τhad.start_cutflow(name='τhad')

            if event.nleps==0:
                Category_τhad.apply_cut('nleps = 0')

                if event.ntaus>=1:
                    Category_τhad.apply_cut('ntaus >= 1')
                    τ=event.tau_leading
                    MET=event.met.pt
                    
                    if MET>150.0:
                        Category_τhad.apply_cut('met > 150')

                        if abs(τ.dphi(event.met))>2.4:
                            Category_τhad.apply_cut('|dphi(τ,met)| > 2.4') 

                            if 0.7<τ.pt/MET<1.3:
                                Category_τhad.apply_cut('0.7 < pT/met < 1.3')
                                mt=τ.MT(event.met)
                                
                                MT_spectrum.extract_spectrum(mt)

                                if mt > mT_min:
                                    Category_τhad.apply_cut('mT > '+str(mT_min))

                                    lhco.parameters['signal_category']=Category_τhad
        lhco.parameters[MT_spectrum.name]=MT_spectrum
        lhco.parameters['MT_bins']=np.array(bins)
        lhco.parameters['effs']=MT_spectrum.bin_data(bins)/float(lhco.parameters['nevents'])

    return lhco.parameters


In [3]:
def extract_results(data_files):

    FF={'FS':'scalar','FT':'tensor','FST':'scalar-tensor',
        'FVLL':'vector LL','FVLR':'vector LR','FD':'dipole','FV':'SM'}      
    
    kernel={}
    
    for lhco in glob.glob(data_files, recursive=True):        
        print('analyzing: '+lhco)
        res=ta_vt_ATLAS_search(lhco, mT_min=200)
        info=res['run_tag'].split('_') 
        kernel[res['run_tag']]={}        
        kernel[res['run_tag']]['Form_Factor']=FF[info[0]]
        if info[1]=='SM':
            kernel[res['run_tag']]['FF_coeff']=None
        else:
            kernel[res['run_tag']]['FF_coeff']=list(info[1])
        kernel[res['run_tag']]['qq~']=info[2]+'~'
        kernel[res['run_tag']]['ll~']=info[3]
        kernel[res['run_tag']]['type']=info[4]+'*'+info[5]
        kernel[res['run_tag']]['mll_bin']=[float(info[-2]),float(info[-1])]
        kernel[res['run_tag']]['results']=res
            
    return kernel

def kernel_matrix(kernel,bins,output):  
    
    def get_bins(bins):
        d={}
        for i,b in enumerate(bins[:-1]):
            I=str([float(b),float(bins[i+1])])
            d[I]=i
        return d
    runs=[] 
    info={}
    for run in kernel.keys():
        x='_'.join(kernel[run]['results']['run_tag'].split('_')[:-2])
        if x not in runs:
            runs.append(x)
            info[x]=kernel[run]
    dic={}
    for key in runs:
        dic[key]={}
    
    bin_number=get_bins(bins)
    
    for r in runs:
        for run in kernel.keys():
            x='_'.join(kernel[run]['results']['run_tag'].split('_')[:-2])
            b=str(kernel[run]['mll_bin'])
            eff=kernel[run]['results']['effs']
            if (kernel[run]['type']=='A*Reg' or
               kernel[run]['type']=='Z*Reg' or
               kernel[run]['type']=='W*Reg'):
                xsec=round(kernel[run]['results']['xsec'],5)
            else:
                xsec=round(2*kernel[run]['results']['xsec'],5)
            pre=[str(xsec),
                 str(kernel[run]['mll_bin'][0]),
                 str(kernel[run]['mll_bin'][1])]
            
            eff_str=''
            
            for y in eff: eff_str+=str(y)+'\t'            
            dic[x][bin_number[b]]='  '.join(pre+[eff_str]) 
    
#     failed_dir=make_dir(output+'/failed_runs_'+, overwrite=True)

    for d in dic.keys():
        file=open(output+'/'+d+'.dat','w')
        file.write('#===============\n')
        file.write('# ll~ : '+ info[d]['ll~'] +'\n')
        file.write('# qq~ : '+ info[d]['qq~'] +'\n')
        ff=info[d]['Form_Factor'].split()
        file.write('# FF  : '+ ff[0] +'\n')
        
        # XY:
        if len(ff)==1:    
            file.write('# XY  : LL LR RL RR\n')
        else:
            if info[d]['Form_Factor'].split()[1]=='LL':
                file.write('# XY  : LL RR  \n')
            elif info[d]['Form_Factor'].split()[1]=='LR':
                file.write('# XY  : LR RL  \n')  
                
        if info[d]['FF_coeff']:
            coeff=info[d]['FF_coeff'][0].replace("'", "")+' '+info[d]['FF_coeff'][1].replace("'", "")
        else:
            coeff='None'
        file.write('# coef : '+coeff+'\n')
        file.write('# type : '+info[d]['type']+'\n')
        file.write('# Nevs : '+ str(info[d]['results']['nevents'])+'\n')
        file.write('#===============\n'  )
        file.write('#\n'  )
        file.write('# XSEC\tBIN_MIN\tBIN_MAX \t Kij \n')
        file.write('#\n'  )
                
        for i in list(bin_number.values()):
            try:
                file.write(dic[d][i]+'\n')
            except KeyError:
                inv_bin_number = {v: k for k, v in bin_number.items()}
                print('Error -->',d,':',inv_bin_number[i])
                file.write((str(None)+'\t')*len(bin_number)+'\n')
                
                # generated failed run:
                
                x=d.split('_')
                xdic={'mumu':'mu_mu', 'tata':'ta_ta', 'ee':'e_e', 
                      'muvm':'mu_vm', 'tavt':'ta_vt', 'eve':'e_ve',
                      'eta':'e_ta', 'tae':'ta_e','muta':'mu_ta',
                      'tamu':'ta_mu','emu':'e_mu','mue':'mu_e'}    
                bin_i=str(bins[i])+'_'+str(bins[i+1])
                x=['PROC']+x+[bin_i]
                qq=x[3]
                x=x[:3]+list(x[3][0])+x[4:]
                run_path='run_scripts/scripts_'+xdic[x[4]]+'/scripts_SMEFT_'+xdic[x[4]]+'_'+bin_i+'/'+'_'.join(x)
                
                new_file=open(output+'/scripts_failed_runs/'+'_'.join(x),'w')
                
                with open(run_path) as f:
                    for line in f:  
                        if '# run_'+qq in line:
                            new_file.write(line)
                        elif '# run_' in line:
                            continue
                        else:
                            new_file.write(line)
                new_file.close()
        file.close()

# def extract_results(data_files):

#     FF={'FS':'scalar','FT':'tensor','FST':'scalar-tensor',
#         'FVLL':'vector LL','FVLR':'vector LR','FDq':'dipole'}      
    
#     kernel={}
    
#     for lhco in glob.glob(data_files, recursive=True):        
#         print('analyzing: '+lhco)
#         res=MonoTau_ATLAS_search(lhco, mT_min=200)
#         info=res['run_tag'].split('_') 
#         if 'int' in info: typ='SM*NP'
#         else: typ='NP*NP' 
#         if '00' in info:
#             qqbar=res['run_tag'].split('_00_')[1].split('_')[0]+'~'
            
#         elif '10' in info:
#             qqbar=res['run_tag'].split('_10_')[1].split('_')[0]+'~'  
            
#         elif '01' in info:
#             qqbar=res['run_tag'].split('_01_')[1].split('_')[0]+'~'
        
#         kernel[res['run_tag']]={}        
#         kernel[res['run_tag']]['Form_Factor']=FF[info[0]]
#         kernel[res['run_tag']]['FF_coeff']=list(info[1])
#         kernel[res['run_tag']]['qq~']=qqbar
#         kernel[res['run_tag']]['ll~']=res['run_tag'].split('_00_')[1].split('_')[1]
#         kernel[res['run_tag']]['type']=typ
#         kernel[res['run_tag']]['pT_bin']=[float(info[-2]),float(info[-1])]
#         kernel[res['run_tag']]['results']=res
    
#     return kernel


In [4]:
t0 = time.time()

FS_kernel  = extract_results(data_files='lhco_files/FS_lhco/*.gz')
FST_kernel = extract_results(data_files='lhco_files/FST_lhco/*.gz')
FT_kernel  = extract_results(data_files='lhco_files/FT_lhco/*.gz')
FVLL_kernel  = extract_results(data_files='lhco_files/FVLL_lhco/*.gz')
FD_kernel  = extract_results(data_files='lhco_files/FD_lhco/*.gz')
FV_kernel  = extract_results(data_files='lhco_files/FV_lhco/*.gz')

t1 = time.time()

print('time =',(t1-t0)/3600.)

analyzing: lhco_files/FS_lhco/FS_00_dc_tavt_Reg_Reg_500_600_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_dc_tavt_Reg_Reg_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_bu_tavt_Reg_Reg_500_600_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_su_tavt_Reg_Reg_300_400_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_ub_tavt_Reg_Reg_500_600_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cs_tavt_Reg_Reg_800_900_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_sc_tavt_Reg_Reg_700_800_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cb_tavt_Reg_Reg_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_du_tavt_Reg_Reg_1500_1750_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_sc_tavt_Reg_Reg_500_600_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cb_tavt_Reg_Reg_200_300_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_su_tavt_Reg_Reg_1500_1750_delphes_events.lhco.g

analyzing: lhco_files/FS_lhco/FS_00_ud_tavt_Reg_Reg_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cd_tavt_Reg_Reg_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_su_tavt_Reg_Reg_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_bu_tavt_Reg_Reg_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cs_tavt_Reg_Reg_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cs_tavt_Reg_Reg_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_du_tavt_Reg_Reg_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cd_tavt_Reg_Reg_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_bc_tavt_Reg_Reg_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cd_tavt_Reg_Reg_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_us_tavt_Reg_Reg_300_400_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_bc_tavt_Reg_Reg_900_1000_delphes_event

analyzing: lhco_files/FST_lhco/FST_00_sc_tavt_Reg_Reg_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_ub_tavt_Reg_Reg_1200_1300_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_cb_tavt_Reg_Reg_200_300_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_sc_tavt_Reg_Reg_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_sc_tavt_Reg_Reg_500_600_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_dc_tavt_Reg_Reg_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_us_tavt_Reg_Reg_1400_1500_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_bu_tavt_Reg_Reg_700_800_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_dc_tavt_Reg_Reg_700_800_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_bc_tavt_Reg_Reg_300_400_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_du_tavt_Reg_Reg_300_400_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_su_tavt_Reg_Reg_13

analyzing: lhco_files/FST_lhco/FST_00_sc_tavt_Reg_Reg_1200_1300_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_dc_tavt_Reg_Reg_1200_1300_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_us_tavt_Reg_Reg_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_bc_tavt_Reg_Reg_1400_1500_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_su_tavt_Reg_Reg_700_800_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_bu_tavt_Reg_Reg_1750_2000_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_su_tavt_Reg_Reg_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_sc_tavt_Reg_Reg_300_400_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_du_tavt_Reg_Reg_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_ud_tavt_Reg_Reg_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_cd_tavt_Reg_Reg_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/FST_lhco/FST_00_cb_tavt_Re

analyzing: lhco_files/FT_lhco/FT_00_cb_tavt_Reg_Reg_800_900_delphes_events.lhco.gz
analyzing: lhco_files/FT_lhco/FT_00_cd_tavt_Reg_Reg_200_300_delphes_events.lhco.gz
analyzing: lhco_files/FT_lhco/FT_00_us_tavt_Reg_Reg_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FT_lhco/FT_00_du_tavt_Reg_Reg_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FT_lhco/FT_00_bu_tavt_Reg_Reg_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/FT_lhco/FT_00_dc_tavt_Reg_Reg_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/FT_lhco/FT_00_dc_tavt_Reg_Reg_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FT_lhco/FT_00_ub_tavt_Reg_Reg_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/FT_lhco/FT_00_cd_tavt_Reg_Reg_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/FT_lhco/FT_00_bu_tavt_Reg_Reg_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FT_lhco/FT_00_bu_tavt_Reg_Reg_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FT_lhco/FT_00_su_tavt_Reg_Reg_200_300_delphes_events

analyzing: lhco_files/FVLL_lhco/FVLL_00_cb_tavt_W_Reg_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_cb_tavt_W_Reg_700_800_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_ud_tavt_Reg_Reg_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_du_tavt_W_Reg_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_ub_tavt_W_Reg_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_cs_tavt_Reg_Reg_300_400_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_ub_tavt_W_Reg_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_du_tavt_W_Reg_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_dc_tavt_Reg_Reg_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_dc_tavt_Reg_Reg_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_cs_tavt_Reg_Reg_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_cs_tavt_W_Reg

analyzing: lhco_files/FVLL_lhco/FVLL_00_ub_tavt_Reg_Reg_1200_1300_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_ud_tavt_W_Reg_1500_1750_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_sc_tavt_W_Reg_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_cb_tavt_W_Reg_200_300_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_sc_tavt_Reg_Reg_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_du_tavt_W_Reg_300_400_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_cs_tavt_Reg_Reg_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_cs_tavt_Reg_Reg_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_ub_tavt_W_Reg_300_400_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_cs_tavt_W_Reg_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_us_tavt_Reg_Reg_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/FVLL_lhco/FVLL_00_ud_t

analyzing: lhco_files/FV_lhco/FV_SM_dd_tavt_W_W_1500_1750_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_ss_tavt_W_W_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_ss_tavt_W_W_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_bb_tavt_W_W_500_600_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_cc_tavt_W_W_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_bb_tavt_W_W_700_800_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_cc_tavt_W_W_300_400_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_dd_tavt_W_W_800_900_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_ss_tavt_W_W_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_cc_tavt_W_W_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_cc_tavt_W_W_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_uu_tavt_W_W_200_300_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_dd_tavt

In [9]:
def kernel_matrix(kernel,bins):  
    
    def get_bins(bins):
        d={}
        for i,b in enumerate(bins[:-1]):
            I=str([float(b),float(bins[i+1])])
            d[I]=i
        return d

    runs=[]    
    for run in kernel.keys():
        x='_'.join(kernel[run]['results']['run_tag'].split('_')[:-2])
        if x not in runs:
            runs.append(x)   
    dic={}
    for key in runs:
        dic[key]={}
    
    bin_number=get_bins(bins)
     
    for r in runs:
        for run in kernel.keys():
            x='_'.join(kernel[run]['results']['run_tag'].split('_')[:-2])
            b=str(kernel[run]['pT_bin'])
            eff=kernel[run]['results']['effs']
            eff_str=''
            for y in eff: eff_str+=str(y)+'\t'
            dic[x][bin_number[b]]=eff_str
                       
    for d in dic.keys():
        file=open('results/'+d,'w')
        for i in list(bin_number.values()):
            try:
                file.write(dic[d][i]+'\n')
            except KeyError:
                inv_bin_number = {v: k for k, v in bin_number.items()}
                print('Error -->',d,':',inv_bin_number[i])
                file.write((str(None)+'\t')*len(bin_number)+'\n')
        file.close()


In [12]:
bins=[200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1750, 2000, 10000]

kernel_matrix(FS_kernel,bins)
kernel_matrix(FT_kernel,bins)
kernel_matrix(FST_kernel,bins)
kernel_matrix(FVLL_kernel,bins)
kernel_matrix(FD_kernel,bins)

Error --> FS_00_cb_tanu : [500.0, 600.0]
Error --> FS_00_cb_tanu : [1400.0, 1500.0]
Error --> FS_00_cd_tanu : [1400.0, 1500.0]
Error --> FS_00_bu_tanu : [1200.0, 1300.0]
Error --> FS_00_bu_tanu : [1400.0, 1500.0]
Error --> FS_00_bu_tanu : [1750.0, 2000.0]
Error --> FS_00_dc_tanu : [1400.0, 1500.0]
Error --> FS_00_du_tanu : [1400.0, 1500.0]
Error --> FS_00_bc_tanu : [500.0, 600.0]
Error --> FS_00_bc_tanu : [1200.0, 1300.0]
Error --> FS_00_bc_tanu : [1300.0, 1400.0]
Error --> FS_00_bc_tanu : [1400.0, 1500.0]
Error --> FS_00_bc_tanu : [1500.0, 1750.0]
Error --> FS_00_bc_tanu : [1750.0, 2000.0]
Error --> FS_00_cs_tanu : [500.0, 600.0]
Error --> FS_00_cs_tanu : [1400.0, 1500.0]
Error --> FST_00_cb_tanu : [500.0, 600.0]
Error --> FVLL_00_bc_tanu_int : [1300.0, 1400.0]
Error --> FVLL_00_cb_tanu : [1300.0, 1400.0]
Error --> FVLL_00_sc_tanu_int : [1300.0, 1400.0]
Error --> FVLL_00_dc_tanu : [1300.0, 1400.0]
Error --> FVLL_00_cs_tanu_int : [1300.0, 1400.0]
Error --> FVLL_00_sc_tanu : [1300.0, 14